In [1]:
import pandas as pd
from collections import defaultdict
from surprise.prediction_algorithms import knns
from surprise.similarities import cosine, msd, pearson
from surprise import accuracy
from surprise import Dataset
from surprise.model_selection import train_test_split, KFold
from surprise import Dataset, Reader
from surprise import Dataset, SVD,SVDpp, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV

In [2]:
df=pd.read_csv('data.csv', index_col=[0])

In [3]:
df.head()

,Beers,Brewery,abv,Url,Id,Rater,From,Rating,Id_beer
0,Traditional Lager,Yuengling Brewery,4.50,/beer/profile/182/1351/,1311021,Derptiluderp,from Indiana,1.23,1
1,Traditional Lager,Yuengling Brewery,4.50,/beer/profile/182/1351/,736885,DCH,from New York,3.99,1
2,Traditional Lager,Yuengling Brewery,4.50,/beer/profile/182/1351/,1310518,md3kcn,from Ohio,2.46,1
3,Traditional Lager,Yuengling Brewery,4.50,/beer/profile/182/1351/,1301271,RobertB412,from Pennsylvania,3.92,1
4,Traditional Lager,Yuengling Brewery,4.50,/beer/profile/182/1351/,1309542,BarfyMan362,from Indiana,4.78,1


In [4]:
df=df[['Rater', 'Beers', 'Rating']]

In [5]:
# Instantiate a 'Reader' to read in the data so Surprise can use it
reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[['Rater', 'Beers', 'Rating']], reader)

In [6]:
trainset = data.build_full_trainset()

In [7]:
trainset

In [8]:
print("Number of users:", trainset.n_users, '\n')
print("Number of items:", trainset.n_items, '\n')

Number of users: 9706 

Number of items: 5071 



There are clearly way more users than items! We'll take that into account when inputting the specifications to our similarity metrics. Because we have fewer items than users, it will be more efficient to calculate item-item similarity rather than user-user similarity.

Now, compare the different models and see which ones perform best. For consistency sake, use MAE to evaluate models

In [9]:
benchmark = []
# Iterate over all algorithms
for algorithm in [SVD(),SVDpp(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore()]:
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['MAE'], cv=3, verbose=False)
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


In [10]:
cross_val=pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_mae')  
cross_val

,test_mae,fit_time,test_time
Algorithm,,,
SVDpp,0.404028,32.196718,1.260894
KNNBaseline,0.418070,2.553445,0.967223
SVD,0.420820,3.148179,0.238999
KNNBasic,0.458189,1.986253,0.788384
KNNWithZScore,0.462440,4.045139,2.260817
KNNWithMeans,0.466108,2.726193,1.238966


Base off these output, SVDpp was the best model with test_mae : 0.404 but its computation time was worst. Therefore, I decided to choose KNNBaseline for this dataset. 

# GridSearch

In [11]:
params = {'n_factors': [20, 50, 100],
         'reg_all': [0.02, 0.05, 0.1]
        }

In [12]:
g_s_knnb = GridSearchCV(KNNBaseline,param_grid=params,n_jobs=-1)
g_s_knnb.fit(data)

In [13]:
print(g_s_knnb.best_score)
print(g_s_knnb.best_params)

{'rmse': 0.5549237314706797, 'mae': 0.4073298202549844}
{'rmse': {'n_factors': 20, 'reg_all': 0.02}, 'mae': {'n_factors': 20, 'reg_all': 0.02}}


In [45]:
sim_options = {'name': 'cosine',
               'user_based': False  # compute  similarities between *items*
               }
bsl_options={'n_factors': 20, 'reg_all': 0.02}
model=KNNBaseline(bsl_options=bsl_options, sim_options=sim_options)
model.fit(trainset)

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.


# Making simple predictions

In [46]:
model.predict(75,500)

Prediction(uid=75, iid=500, r_ui=None, est=3.4029386313281216, details={'was_impossible': False})

# Item to Item Recommendation

In [17]:
model.trainset.to_inner_iid("Traditional Lager")

0

In [18]:
def beerrec(beer, k=5):
    """
    This function takes in a name of a beer and gives you k recommendations for other beers that you may like
    """
    iid = model.trainset.to_inner_iid(beer) #changes the name of the beer to its id
    
    recs = model.get_neighbors(iid, k) #takes the id and gives you k recommendations for it (also in id form)
    
    for rec in recs:
        print(model.trainset.to_raw_iid(rec)) #takes the id and converts that back into a beer name and prints it

In [19]:
beerrec('Lager')

Cool Ridge Lager
Saranac 1888 Octoberfest
Club Colombia Roja
Wisconsin Amber
DAB Dark Beer


In [20]:
beerrec("Traditional Lager")

Original Honey Brown Extra Rich Lager (Beer Brewed With Honey)
Point Classic Amber
George Killian's Irish Red
Third Shift Amber Lager
Riverwest Stein Beer


# Taking top 10 recommend beers

In [25]:
trainset, testset=train_test_split(data, test_size=0.2)

In [26]:
#This code below, I copied from surprise library document

In [27]:
from collections import defaultdict

from surprise import SVD
from surprise import Dataset


def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


# First train an SVD algorithm on the movielens dataset.
#data = Dataset.load_from_df(df[['Rater', 'Beers', 'Rating']], reader)
#trainset = data.build_full_trainset()
algo = model
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
#testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
kjyost ['1516 Bavarian Lager', 'Nikšićko Pivo']
Brent ['Organic Real Lager', 'Aass Juleøl', 'Thunder Road', 'Laško Club', 'Hurricane Reef Caribbean Pilsner', 'Quisqueya Extra', 'Mountain Light', 'Fischer LaBelle']
jgido759 ['Pilsner', 'Samuel Adams Coffee Black Lager']
jlindros ['The Standard', 'Koutská 12° Dvanáctka', 'Ratskeller', 'Landbier Dunkel', 'Boston Rising', 'Riverton Flyer', 'Modernism', 'Luminaire', 'Deep Purple Rauchbier', 'Emelyn']
davidperez ['Kaiserdom Kellerbier Unfiltriert']
LoveStout ['Red Wing Lager']
champ103 ['Dortmunder', 'Old Pavilion Pilsner', 'Guten Tag', 'Behemoth Pilsner', 'Calavera Lager', '(512) Export Lager', 'Dortmunder Mifflin']
RaulMondesi ['Andechser Vollbier Hell', 'Reality Czeck', 'The Cottage', '1903', 'Pils', 'Coastal Common', 'Oscura', 'Pony', 'Spaten Oktoberfestbier', 'Copper Legend Octoberfest']
kylehay2004 ['Dundee Irish Red Lager', 'Conchto

kduke42 ["Heileman's Special Export"]
mikeg67 ['Organic Winter Spezial', 'Rauchbier', 'Short Fuse', 'Mahou Maestra Dunkel', 'Post Shift Pilsner', 'Hoffmann Oktoberfest', 'Vienna Red Lager', 'Dark Lager', 'Wolters Pilsener', 'Bohemia Especial']
crenegar ['Building Blocks', 'Ocean State Lager', 'Hackensack Lager']
TheGent ['Penn Dark', 'Mariestads Old Ox', 'Carlsberg Sort Guld']
donspublic ['Bat Outta Helles', 'Shiner Sea Salt & Lime Summer Lager', 'Shiner 777 IPL', 'Magic Flip', 'Bavik Super Pils', 'Budweiser 1933 Repeal Reserve Amber Lager', 'Imperium Premium Belgium Lager']
Grogsky ['Cracked Canoe']
soju6 ['Yebisu Creamy Top Stout', 'Pfungstädter Export', 'Little Creatures Original Pilsner', 'Premium Export Lager', 'Steinlager', 'Schultheiss Pilsner', 'Bluetongue Premium Lager', 'Maes Pils', 'Leglifter Light', 'XXXX Gold']
MikeyA ['Coney Island Lager']
Vetbidder ['Tropic Of Thunder', 'Meltwater India Pale Lager', 'Oktoberfest', 'Pilsner', 'Samuel Adams Noble Pils', 'Kenzinger', 'Old G

Abflex84 ['Colorado Native IPL']
GratefulBeerGuy ['Ella', 'Trailblazer', 'Weed Lemurian Golden Lager', 'Fresh Cut', 'PILS', 'Helles Lager', 'Bella Czech Pils', 'Tuberfest', 'Premium Lager', 'Single Speed IPL']
animal69 ['Labatt Genuine Honey', 'Schlitz Red Bull X. L. Xtra Long Malt Liquor']
Blakaeris ['Podkovan Lager', 'B.B. Dark Bohemia Beer - 1795 Original Czech Dark Lager', 'Bahia Lager']
UncleJimbo ['Klosterbräu Schwärzla', 'Witzgall Lagerbier (Schlammersdorfer Landbier)', 'Fischer Rauchbier', 'Wagner Ungespundetes Lagerbier', 'Pilsner', 'Munich Gold', 'El Inca']
BlesstheHarvest ['Big Bark Amber Lager']
Thebrewman ['Samuel Adams Downtime Pilsner']
BavariaYeti ['Pilsner']
mnj21655 ['Red Tape Lager', 'Remain In Light']
Eamonn-Cummings3 ['Dark Knife']
beer2day ['Oktoberfest Lager']
CsRed ['Budweiser Select']
theopholis ['Bamberger Herren Pils', 'Columbus Pils', 'Efes Pilsener Fici/Draft', 'Wild Hop Organic Amber Lager', "Beck's Light"]
MirkoCroCough ['Premium Light Beer', 'American Li

SmartyPints ['Colt 444']
Dawkfan ['Raasted Vinter', 'Czech Mate']
facundoCNB ['Elsa Von Horizon', 'Before-After Premium Lager (12% ABV)']
wiscokid920 ['Garten Bräu Special Pilsner', 'O-toberfest']
Imbiber ['Tusker Malt Lager', 'Rycerskie', 'Birra Tirana']
Ohiovania ['Moravian Rhapsody', '1872 Pre-Prohibition Lager']
D-Rock ['Sapporo Classic']
lochef81 ['UnderDog Atlantic Lager']
UnionMade ["Pott's Landbier", 'Oktoberfest Lager', 'Vyskovske Pivo', 'Yinpu Black Rice Beer']
tconboy ["Pott's Landbier"]
Biscuit033 ['Victory Dark Lager']
billrustin ['Anker Bir']
oN3xM ['SOS (Save Our Shore)']
Alic099 ['Corona Extra']
OzmanBey ['Life On Märzen', 'O.K. Beer', 'Amstel']
Bugs318 ['Starke Pilsner']
foamyhead ['Maccabee Premium Beer']
Rogerlb ['Únětické Pivo 12°', 'Ottakringer Helles', 'Super Bock']
doubleg35 ['Pabst Blue Ribbon']
pastormc93 ['Royal Dutch Post Horn 16%', 'San Miguel Super Dry']
Blackop555 ['Pilz', 'Big Bark Amber Lager', 'Tidal Lager', 'San Miguel Pale Pilsen']
wspscott ['Tap Room

Brewsome ['Onshore Lager']
Aerocraft67 ["Saranac F.X. Matt's Haus Lager"]
Becki4Beers ['Two Women Lager']
Shane1959 ['Carlton Draught']
Nevbo ['KCCO Gold Lager']
DoubleMalt ['Cold Brew Coffee Lager']
tayner ['Baro Tradicinis']
bluegrassbrew ['Columbus Pils']
Scottync ['Red Oak']
Alkey ['Märzen', 'Oktoberfest']
Noxious26 ['Gold Extra']
nealtor57 ['Keystone Light']
Thrasher ['Brok Export']
Riff ['NightPils', 'Hardywood Pils', "Lil' Hellion", 'Witching Hour Black Lager', "Joseph's Brau Bohemian Lager"]
joedvegas ['Pilsner']
mpcrump ['Preservation Pilsner']
DAllspaw ['Brown Lager']
mndlssphlsphr ['Oranjeboom Super Strong', 'Oranjeboom Extra Strong']
vfgccp ['Binding Römer Pils', 'Brouček (BrouCzech Lager)', 'KEO Beer']
cpanossian ['Lösch-Zwerg Würzig']
ferz ['Bat Outta Helles']
beerhobo22 ['Lager', 'Golden Export']
wattoclone ['Saranac Our Octoberfest German Style Lager', 'Cold Brew Coffee Lager', 'I Dunkeled In My Pants', 'Pennsylvania Style Light Beer']
allbeersgood ['Vibin’']
YoungMtn52

dave_ackbar ['Samuel Adams Noble Pils', 'Hite Extra Cold']
ArcMage ['Estaminet Premium Pils']
Socialiste ['Boris Slam']
Hobo53527 ['Chick Beer']
jslice99 ['Hurricane Malt Liquor']
FxLB ['Simpler Times Lager']
haharight ['Carling Black Label']
Rudgers73 ['Slavutich Mitzne', 'Milwaukee Premium Brewing Company Classic Light']
lordsellers04 ['Hirter Privat Pils']
RobbieAshmore ['Point Oktoberfest']
Whiteweasel ['Session Lager']
osheamatth ['Ok2berfest']
fostwal ['Rauchbier']
DenverLogan ['Olde Frothingslosh']
TonyLema1 ["Joe's Premium American Pilsner"]
henrysunset ['Schwartzeit']
leonardopacu ['Cerpa Export']
deereless ['Bareville Pilsner']
LordFoul ['Mecktoberfest']
Delraypete ['Stiltsville Pilsner']
Beers-es ['Prima Pils']
flexabull ['Imperial Pilsner']
BigRedDog ['Rauch Ür Bock']
OneDropSoup ['Staropramen Lager (Le\x9eák)']
likestoswallow ['Mojave Red']
istarilord ['Hahn Premium']
interzen ['Gypsy Lager', 'Vedett Extra Blond']
hoppydanknessOG ['Red Oak']
JoeyBeerBelly ['Traditional Lag

Schwarzbart ['Oktoberfest']
WJVII ['Oktoberfest']
Savan ['Staghorn Octoberfest']
MiguelEstrada ['Indio']
heinekenike ['Wieselburger Stammbräu']
diabeticbeerman ['The Purist']
watermelon ['I.C. Light']
branko7 ['Paracelsus Zwickl', 'Norrlands Guld Export']
beermanbrian ['Peregrine Pilsner']
brobbable ['Iron City Beer']
Crimson-RV ['Tiny Bomb', 'Oktober Fuel']
Clydesdale ['Stegmaier Amber Lager']
Slyjoker87 ['Love Rock']
Theheroguy ['Ettaler Kloster Dunkel']
fox227 ['Neumarkter Lammsbräu (Organic) Pilsner']
atrocity ["Doctor's Orders"]
Mellonius ['Super Bock Abadia']
Notallan ["Stroh's"]
abastias ['Loose Caboose Lager']
Seanibus ['Gila Monster Amber Lager']
FightingEntropy ["5 O'Clock Shadow"]
Myboymi ['Märzen Oktoberfest']
TheMarkE ['Steinlager', 'American Patriot']
orbitalr0x ['Bolt 117 Lager']
psugrad98 ['American Light']
beerloserLI ['Prima Pils']
scootny ['Broken Auger Lager']
JackWeizen ['Copper Flask']
Contagion ["Henry Weinhard's Blonde"]
SayBoom ['Pfungstädter Urstoff Kellerbier

In [28]:
# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

kjyost ['1516 Bavarian Lager', 'Nikšićko Pivo']
Brent ['Organic Real Lager', 'Aass Juleøl', 'Thunder Road', 'Laško Club', 'Hurricane Reef Caribbean Pilsner', 'Quisqueya Extra', 'Mountain Light', 'Fischer LaBelle']
jgido759 ['Pilsner', 'Samuel Adams Coffee Black Lager']
jlindros ['The Standard', 'Koutská 12° Dvanáctka', 'Ratskeller', 'Landbier Dunkel', 'Boston Rising', 'Riverton Flyer', 'Modernism', 'Luminaire', 'Deep Purple Rauchbier', 'Emelyn']
davidperez ['Kaiserdom Kellerbier Unfiltriert']
LoveStout ['Red Wing Lager']
champ103 ['Dortmunder', 'Old Pavilion Pilsner', 'Guten Tag', 'Behemoth Pilsner', 'Calavera Lager', '(512) Export Lager', 'Dortmunder Mifflin']
RaulMondesi ['Andechser Vollbier Hell', 'Reality Czeck', 'The Cottage', '1903', 'Pils', 'Coastal Common', 'Oscura', 'Pony', 'Spaten Oktoberfestbier', 'Copper Legend Octoberfest']
kylehay2004 ['Dundee Irish Red Lager', 'Conchtoberfest', 'Aruba Red', 'Festie Oktoberfest Lager (Heavy Rotation Series No. 07)', 'Amstel Bright']
stevo

LampertLand ['Vienna Amber Lager', 'Sapporo Premium Beer', 'Premium German Pilsener', 'Labatt Blue']
IronDjinn ['Raasted Vinter', 'Rousse', 'Sherbrooke Shamrock', 'Alligator Blackened Lager', 'Yellowhead Premium Lager', 'Powder Hound Pilsner', 'Lech Premium', 'Millennium Buzz Hemp Beer', 'Cool Beer', 'Pacific Pilsner']
JustinBathurst ['Hell']
SoccerSeuss ["Lil' Hellion"]
Mick_Bynes ['Menabrea Original']
Bestuurder ['PC Honey Red']
spointon ['Imperial Pilsner']
swim ['Trailer Park Boys Freedom 35', 'Laker Red', 'Tuborg Premium Biere De Luxe Gold']
pat61 ['The Wright Pils', 'Keller Pils', 'Pau Hana Pilsner', 'Munsterfest', 'Stargrazer', 'Pilsner', 'Black Forest Pils', 'Winter Lager', 'Summarillo', 'Premium Pils']
Globetrotter ['Černovar Černé', 'Radegast Premium', 'Öko Urtrunk', 'Yarpivo Originalnoe', 'Zlatopramen 11°', 'Bernard Světlý Ležák 11°', 'Skopsko', 'Allgäuer Ökobier', 'Kaiser Fasstyp', 'Efes Pilsener Fici/Draft']
jwc215 ['Ramstein Revelation Golden Lager', 'Pils', 'Oktoberfest'

Converge ['Gold Strong']
Andrewharemza ['König Ludwig Dunkel', 'Budweiser Budvar Czech Dark Lager', 'Fathom IPL', 'Altenmünster Urig Würzig', 'Pistonhead Kustom Lager', 'Cerveza Cristal', 'Desperados', 'Cascade Premium Light', "James Boag's Premium Light", 'Bighead']
raznug ['Mitternacht German Style Black Ale', 'Barnburner Lager', 'Dunkel']
rudzud ['Poetica 3', 'Memoria', 'Rhetorica', 'Altoberfest', 'Černé Zámecké', 'Coffee Smoke & Dagger', 'Starlight Lager', 'Lomza Pelne (Premium Beer)', 'Coors Extra Gold', 'Bomonti']
ehreesejr ['Iron Hill Light Lager']
bnietz ['Grain Belt Premium Light']
Treyliff ['Oktoberfest', 'Trösten', 'Altenmünster Oktoberfest', 'Double Black Lager', 'Oktoberfest Bier (Oktoberfest Wiesn)', "Devil's Lettuce", 'Dunkel', 'Fest Märzen Lager', 'Premium Pils (Premium Beer)', 'KillaPilz']
Metalmonk ['Bobtoberfest', 'OktoberBeast', 'Lev Lion Lager', 'Odenwalder Schwarzbier']
pro45 ['914']
Stevosps ['Oktoberfest']
end374 ['Big Flats 1901']
aleigator ['Qualify Pils', 'Ri

Jbbdrinker ['Oktoberfest', 'Märzen', 'Praga Dark Lager', 'Shiner Premium', 'Nowhere But Texas', 'Cowboys from Helles', 'Kronenbourg 1664', 'Neato Bandito', 'Deep Ellum Lager', 'Accused Amber Lager']
mintjellie ['Millennium Buzz Hemp Beer', 'Carling Ice', 'Carling Beer']
TX-Badger ['Reasonably Corrupt', 'Mother In Lager', 'Saturday Siren Dry-Hopped Pilsner', 'Kato Lager Beer', 'Helles Belles', 'Golden Mustache', 'LA-31 Bière Noire', 'Octoberfest (Märzen-Style) Lager', 'Dream Machine', 'Premium Lager']
Anders91 ['Oktoberfest Bier (Oktoberfest Wiesn)', 'Hacker-Pschorr Oktoberfest Märzen', 'STS Pils', 'Spaten Oktoberfestbier', 'Weltenburger Kloster Anno 1050', 'Hofbräu Oktoberfestbier']
carln26 ['Summer Beer', 'Oktoberfest Lager']
silverking ['Weissenoher Bonifatius Dunkel']
buschbeer ['Kaiser Pils', 'Staghorn Octoberfest', 'Feel Feelings', 'Primator Dark Lager', 'Harp Lager', 'Cerveza De Julie', 'Leikeim Kellerbier Naturtrüb', 'Natty Rush Hurricane Punch (12%)', 'Natty Rush Watermelon Sma

TheDominic ['Schlitz High Gravity V.S.L Very Smooth Lager', 'King Cobra Premium Malt Liquor']
Jayli ['Newport Storm Oktoberfest Marzen Lager', 'Black Licorice Lager', 'Polar Pilsen']
dfried ['Burntwood Black Ale']
SAtaMi ['Pabst Blue Ribbon Light']
the_awesometeer ['Elephant Beer', "Trader Jose's Dark Lager", 'Baltika #7 Export']
Ri0 ['Pils', 'Vienna Lager', '#010 Patron Saint']
aatazebeats ['Baltika #2 Svetloye', 'Frio 6.0', 'Icehouse']
Ricko ['Coors Extra Gold']
moodenba ['Post Shift Pilsner', '8th Sin Black Lager', 'Roly Poly Pils', 'Saratoga Lager', 'Perlenbacher Imperial', "Season's Best Winter Lager", 'Session Lager', 'German Style Lager', 'Perlenbacher Premium Pils']
DokiDokiLitFam ['Coconut Cream Pie']
adams82 ['Schlitz Genuine Gold']
JFear ['Dam Lager']
BRONsnthm ['Barmen Pilsner']
WDJersey ['Bamberger Rauchbier', 'Mockro Lager']
TommyW ['Bockbier', 'Temple Bräu']
Sludgeman ['Weltenburger Kloster Barock Dunkel', 'Spezial Pils', 'Pearl-Snap', 'Obolon Oksamytove (Deep Velvet)']


Fuldner ['Weihnachtsbier', 'Berliner Pilsner']
scruffwhor ['Miner Mishap', 'Zagloba Mocne', 'Herold Traditional Premium Bohemian Lager', 'Bernard Světlý Ležák 11°', 'Lomza Mocne', 'Neuzeller Kloster-Bräu Schwarzer Abt (Black Abbot)']
rwarde ['Molson XXX']
EyePeeAyBryan ["Lefty's Lager"]
Beertsipper ['Samuel Adams Wicked Easy', 'Krombacher Pils', 'Modelo Especial', 'Brouček (BrouCzech Lager)']
Matt_Rouge ['Jinx Proof']
oline73 ['Pious Monk Dunkel', 'Dark Helmet Imperial Schwarzbier']
WhiteHillsStore ['Oktoberfest Märzen Style Lager']
wmscottsimpsonjr ['Ayinger Oktober Fest-Märzen', 'Stammtisch', 'Technical Ecstasy', 'Amadeus Vienna Lager', 'Dorfbier', 'Oktoberfest']
holder1422 ['Lightswitch Lager']
Dray ['I.C. Light Mango']
nunitn ['Sapporo Premium Black Beer']
ZionNation ['Elemental Pilsener']
flocculate ['Continental Lager']
alpinebryant ['Ybor Gold Amber Lager']
DanielB93 ['Cerveza Pacifico Clara']
OompaMentor ['Jenny Lake Lager', 'Coney Island Freaktoberfest']
BFCarr ['Samuel Adams 

matthewarmstrong ['Carlton Cold']
WYVYRN527 ['Grieskirchner Pils']
malinali14 ['Yard Sale Winter Lager']
Holderness ['Helles', 'G.O.A.T.', 'Spezial Pils']
3Vandoo ['Cruzcampo Pilsener']
squasha ['Pils']
RJLarse ['Irish Style Lager']
Alexkauruk ['Zatecky Gus']
jsouza ['Opa’s Oktoberfest']
Col ['Pilsener']
bryehn ['Molson Canadian 67 Sublime']
mintzmeet ['The One They Call Zoe']
mhe1931 ['Leaves Of Grass Series VI: Salut au Monde!']
rho222 ['Angkor Beer']
RomeosDelight69 ['Oktoberfest']
nofalltoofar ['3 Horses Holland Lager Beer']
ottovon ['ML209']
clemtig76 ['Dutch Windmill']
BCMan ['I-Ain-Ga-Lie']
UncleRandy ['Augustiner Bräu Märzen Bier']
stella77artois ['Moosbacher Lager']
madvoice ['Bluetongue Premium Lager']
MacQ32 ['Tripping Brick']
Hhawk06 ['Oktoberfest']
Long813 ['Soproni Fekete Démon']
ZorPrime ['Grafenwalder Strong', 'AC/DC']
bondjedi ['Kokanee Glacier Beer']
CNoj012 ['Mass Rising']
HockeytownBrew ['Kirin Light Beer']
Thorax ['Breznak Schwarzbier']
drinkininthewood ['Sleeman L

JMWEB2 ['Binding Römer Pils']
SaCkErZ9 ['Ettaler Kloster Edel Hell']
Tr0janSlay3r ['IPL']
orfeu ['Oktoberfest Lager']
Sparky44 ['Virunga Mist']
Raven3464 ['Menabrea 1846']
Ncollins ['Redbridge']
ih8rt ['Blonde Lager']
jaluria ['Okocim Jasne Pelne']
BB8Bbbeer ['Hoyner Pilsner']
Breer ['Kozel']
afatty ['Fischer LaBelle']
newk340 ['Summerfest Lager']
Coorsy ['Great Ketch']
blakelive784 ['Winterfest']
kyleelton ['Frio Light']
TSJ4000 ['Leikeim Premium Pils']
Wobbly ['Dunkel']
sergio1va ['Iron Triangle Dark Ale']
jnewm777 ['Bürger Light']
MadLatvian ['Prestige']
chanokokoro ['Magnetron']
BeerDocT ['Rocket Girl']
piss-wreck44 ['Tooheys Extra Dry']
Sorcerer ['Obolon Premium']
feelmefoam ['Tap Room No. 21 Lager']
emmasdad ['FestBier', 'Fest Bier Märzen Lager']
whtguitarguy ['Fischer LaBelle']
JuliusBonsall ['Coors Banquet']
webmanspidey ['Lager']
Tysaison ['Helles']
a77cj7 ['Melon Smash IPL']
AnalogReigns ['Amber Lager']
runningguydc ['Gold Leaf Lager']
7beerphil ['Schaefer Light']
BierFan ['P